# SIROS1DUTT.C1 Plots

[Click here](https://www.arm.gov/capabilities/vaps/diffcor) for more information about this vap.

In [ ]:
%matplotlib widget
import ipywidgets as widgets

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from datetime import datetime

import act
import xarray as xr

# Data archive directory
DATA_DIR = r'/data/archive/'

# Datastream info
DATASTREAM_NAME = 'siros1dutt'
DATA_LEVEL = 'c1'
LOCATIONS = [{'end_date': '1997-10-30', 'facility': 'E10', 'site': 'sgp', 'start_date': '1995-07-21'}, {'end_date': '1997-08-21', 'facility': 'E11', 'site': 'sgp', 'start_date': '1995-06-30'}, {'end_date': '1997-10-26', 'facility': 'E12', 'site': 'sgp', 'start_date': '1996-01-19'}, {'end_date': '1997-08-25', 'facility': 'E13', 'site': 'sgp', 'start_date': '1994-01-07'}, {'end_date': '1997-08-14', 'facility': 'E15', 'site': 'sgp', 'start_date': '1994-01-12'}, {'end_date': '1997-08-20', 'facility': 'E16', 'site': 'sgp', 'start_date': '1995-06-02'}, {'end_date': '1997-09-15', 'facility': 'E18', 'site': 'sgp', 'start_date': '1996-06-20'}, {'end_date': '1997-11-20', 'facility': 'E1', 'site': 'sgp', 'start_date': '1995-11-15'}, {'end_date': '1998-02-10', 'facility': 'E20', 'site': 'sgp', 'start_date': '1994-11-03'}, {'end_date': '1997-11-25', 'facility': 'E22', 'site': 'sgp', 'start_date': '1995-03-16'}, {'end_date': '1997-11-25', 'facility': 'E24', 'site': 'sgp', 'start_date': '1995-11-07'}, {'end_date': '1997-11-05', 'facility': 'E2', 'site': 'sgp', 'start_date': '1996-03-07'}, {'end_date': '1997-11-04', 'facility': 'E3', 'site': 'sgp', 'start_date': '1996-03-06'}, {'end_date': '1997-08-07', 'facility': 'E4', 'site': 'sgp', 'start_date': '1995-05-08'}, {'end_date': '1997-11-06', 'facility': 'E5', 'site': 'sgp', 'start_date': '1996-06-14'}, {'end_date': '1997-11-05', 'facility': 'E6', 'site': 'sgp', 'start_date': '1996-03-05'}, {'end_date': '1997-10-31', 'facility': 'E7', 'site': 'sgp', 'start_date': '1995-05-18'}, {'end_date': '1997-07-14', 'facility': 'E8', 'site': 'sgp', 'start_date': '1995-09-22'}, {'end_date': '1998-02-03', 'facility': 'E9', 'site': 'sgp', 'start_date': '1994-01-12'}]

## Define site, facility, and date ranges

In [ ]:
print("The following locations and date ranges are available for this VAP:")
display(pd.DataFrame(LOCATIONS, columns=['site', 'facility', 'start_date', 'end_date']))

In [ ]:
# Define site, facility, and date range below:
site_facility = ( 'sgp', 'E9' )

# Date range. Format: YYYY-MM-DD
date_start = '1998-01-28'
date_end = '1998-02-02'

## Load data files
Loads data files from /data/archive/

In [ ]:
# Compile list of files
site, facility = site_facility
d_date_start = datetime.strptime(date_start, '%Y-%m-%d')
d_date_end = datetime.strptime(date_end, '%Y-%m-%d')
dir_path = os.path.join(DATA_DIR + site, site + DATASTREAM_NAME + facility + r'.' + DATA_LEVEL )
files_list  = []
for f in os.listdir(dir_path):
    file_date_str = f.split(r'.')[2]
    try:
        file_date = datetime.strptime(file_date_str, '%Y%m%d')
    except Exception:
        continue 
    if d_date_start <= file_date and d_date_end > file_date:
        files_list.append(os.path.join(dir_path, f))

# Load files as a single dataset
ds = act.io.armfiles.read_netcdf(files_list)
ds.clean.cleanup()
print(f'{len(files_list)} files loaded')
ds


## Plot time series data

In [ ]:
# Define the list of variables to be plotted
variables_to_plot = ['lat', 'lon', 'alt']

In [ ]:
ts_display = act.plotting.TimeSeriesDisplay(ds)
ts_display.add_subplots((len(variables_to_plot),), figsize = (13,4*len(variables_to_plot)))

for i,v in enumerate(variables_to_plot):
    ts_ax = ts_display.plot(v, subplot_index=(i,), set_title=ds.variables[v].attrs['long_name'],)
    ts_ax.grid()

plt.show()


## Field selection dropdown menu

In [ ]:
plt.ioff()

# populate dropdown menu with available variables 
available_variables = [v for v in ds.variables if not('time' in v or v.startswith('qc_') or v.startswith('source_')) and 'long_name' in ds.variables[v].attrs]
d_variable = 'lat'
dropdown = widgets.Dropdown(
    options = [(ds.variables[v].attrs['long_name'], v) for v in available_variables],
    value= d_variable,
    description='Field:',
    disabled=False,
)
dropdown.layout.margin = '0px 30% 0px 20%'
dropdown.layout.width = '50%'

# set up display
i_display = act.plotting.TimeSeriesDisplay(ds)
i_display.add_subplots((1,), figsize = (13,5))
i_ax = i_display.plot(d_variable, subplot_index=(0,), set_title=ds.variables[d_variable].attrs['long_name'],)
i_ax.grid()
i_fig = i_display.fig

# update plot callback function
def update_plot(change):
    i_ax.cla()
    i_ax_new = i_display.plot(change.new, subplot_index=(0,), set_title=ds.variables[change.new].attrs['long_name'],)
    i_ax_new.grid()
    i_fig.canvas.draw()
    i_fig.canvas.flush_events()

dropdown.observe(update_plot, names='value')

widgets.AppLayout(
    header=dropdown,
    center=i_fig.canvas,
    pane_heights=[1, 6,1]
)
